# MCQ Generation Fine-tuning with Unsloth GRPO
**Fine-tune Qwen2.5-3B-Instruct to generate Multiple Choice Questions from text context**

This notebook fine-tunes a model to:
- Take text passages as input
- Generate MCQ questions with 4 options and correct answer
- Output structured JSON format

Based on EleutherAI/RACE dataset format

## 1. Installation

In [1]:
import os
os.environ["UNSLOTH_VLLM_STANDBY"] = "1"

# Install dependencies for Modal
!pip install --upgrade -q uv

try:
    import numpy, PIL
    get_numpy = f"numpy=={numpy.__version__}"
    get_pil = f"pillow=={PIL.__version__}"
except:
    get_numpy = "numpy"
    get_pil = "pillow"

import subprocess
try:
    is_t4 = "Tesla T4" in str(subprocess.check_output(["nvidia-smi"]))
except:
    is_t4 = False

get_vllm = "vllm==0.9.2" if is_t4 else "vllm==0.10.2"
get_triton = "triton==3.2.0" if is_t4 else "triton"

!uv pip install -q --system unsloth {get_vllm} {get_numpy} {get_pil} torchvision bitsandbytes xformers
!uv pip install -q --system {get_triton}
!uv pip install --system transformers==4.56.2
!uv pip install --system --no-deps trl==0.22.2
!uv pip install --system pydantic datasets

print("✓ Installation complete!")


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Using Python 3.12.6 environment at: /usr/local
Resolved 18 packages in 57ms
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠹ Preparing packages... (0/1)
⠹ Preparing packages... (0/1)
⠹ Preparing packages... (0/1)
⠹ Preparing packages... (0/1)
⠸ Preparing packages... (0/1)
⠸ Preparing packages... (0/1)
⠸ P

## 2. Load Model

In [1]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 1536  # Reduced from 2048
lora_rank = 64

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-3B-Instruct",
    max_seq_length = max_seq_length,
    load_in_4bit = True,
    fast_inference = True,
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.65,  # Changed from 0.9
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank,
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

print("✓ Model loaded successfully!")


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 11-16 19:40:14 [__init__.py:216] Automatically detected platform cuda.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 11-16 19:40:23 [vllm_utils.py:700] Unsloth: Patching vLLM v1 graph capture
INFO 11-16 19:40:23 [vllm_utils.py:730] Unsloth: Patching vLLM v0 graph capture
==((====))==  Unsloth 2025.11.3: Fast Qwen2 patching. Transformers: 4.56.2. vLLM: 0.10.2.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.251 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu129. CUDA: 8.0. CUDA Toolkit: 12.9. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/qwen2.5-3b-instruct-unsloth-bnb-4bit with actual GPU utilization = 64.63%
Unsloth: Your GPU has CUDA compute capa

`torch_dtype` is deprecated! Use `dtype` instead!


INFO 11-16 19:40:41 [__init__.py:1815] Using max model len 1536
WARNING 11-16 19:40:41 [_ipex_ops.py:16] Import error msg: No module named 'intel_extension_for_pytorch'
INFO 11-16 19:40:42 [scheduler.py:222] Chunked prefill is enabled with max_num_batched_tokens=2048.
WARNING 11-16 19:40:42 [lora.py:92] `lora_extra_vocab_size` is deprecated and will be removed in v0.12.0. Additional vocabulary support for LoRA adapters is being phased out.
Unsloth: vLLM Bitsandbytes config using kwargs = {'load_in_8bit': False, 'load_in_4bit': True, 'bnb_4bit_compute_dtype': 'bfloat16', 'bnb_4bit_quant_storage': 'uint8', 'bnb_4bit_quant_type': 'nf4', 'bnb_4bit_use_double_quant': True, 'llm_int8_enable_fp32_cpu_offload': False, 'llm_int8_has_fp16_weight': False, 'llm_int8_skip_modules': ['lm_head', 'multi_modal_projector', 'merger', 'modality_projection', 'model.layers.2.mlp', 'model.layers.3.mlp', 'model.layers.30.mlp'], 'llm_int8_threshold': 6.0}
INFO 11-16 19:40:44 [core.py:76] Initializing a V1 LLM 

[W1116 19:40:44.414189312 ProcessGroupNCCL.cpp:981] Warning: TORCH_NCCL_AVOID_RECORD_STREAMS is the default now, this environment variable is thus deprecated. (function operator())


INFO 11-16 19:40:44 [gpu_model_runner.py:2370] Loading model from scratch...
INFO 11-16 19:40:45 [cuda.py:362] Using Flash Attention backend on V1 engine.
INFO 11-16 19:40:45 [bitsandbytes_loader.py:758] Loading weights with BitsAndBytes quantization. May take a while ...
INFO 11-16 19:40:45 [weight_utils.py:348] Using model weights format ['*.safetensors']
INFO 11-16 19:40:46 [weight_utils.py:406] No model.safetensors.index.json found in remote.


...

Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|█████████████████| 67/67 [00:21<00:00,  3.14it/s]
Capturing CUDA graphs (decode, FULL): 100%|████████████████████████████████████| 49/49 [00:15<00:00,  3.13it/s]

INFO 11-16 19:41:57 [gpu_model_runner.py:3118] Graph capturing finished in 37 secs, took 2.54 GiB
INFO 11-16 19:41:57 [vllm_utils.py:712] Unsloth: Patched vLLM v1 graph capture finished in 37 secs.


INFO 11-16 19:41:58 [gpu_worker.py:391] Free memory on device (78.72/79.25 GiB) on startup. Desired GPU memory utilization is (0.6462930630816577, 51.22 GiB). Actual usage is 2.44 GiB for weight, 2.01 GiB for peak activation, 0.02 GiB for non-torch memory, and 2.54 GiB for CUDAGraph memory. Replace gpu_memory_utilization config with `--kv-cache-memory=47317980569` to fit into requested memory, or `--kv-cache-memory=76851624960` to fully utilize gpu memory. Current kv cache memory in use is 50199467417 bytes.
INFO 11-16 19:41:59 [core.py:218] init engine (profile, create kv cache, warmup model) took 71.25 seconds
INFO 11-16 19:42:00 [llm.py:295] Supported_tasks: ('generate',)
INFO 11-16 19:42:00 [__init__.py:36] No IOProcessor plugins requested by the model
Unsloth: Just some info: will skip parsing ['attention_norm', 'k_norm', 'pre_feedforward_layernorm', 'post_attention_layernorm', 'input_layernorm', 'ffn_norm', 'post_layernorm', 'norm', 'norm2', 'q_norm', 'layer_norm2', 'layer_norm1'

Unsloth 2025.11.3 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


✓ Model loaded successfully!


## 3. Define Pydantic Output Schema

In [3]:
from pydantic import BaseModel, Field
from typing import List
import json

class MCQQuestion(BaseModel):
    """Single Multiple Choice Question"""
    question: str = Field(description="The question text")
    option_a: str = Field(description="Option A")
    option_b: str = Field(description="Option B")
    option_c: str = Field(description="Option C")
    option_d: str = Field(description="Option D")
    correct_answer: str = Field(description="Correct answer: A, B, C, or D")

class MCQList(BaseModel):
    """List of MCQ Questions"""
    questions: List[MCQQuestion] = Field(description="List of generated questions")

print("Expected JSON Output Format:")
print(json.dumps(MCQList.model_json_schema(), indent=2))

Expected JSON Output Format:
{
  "$defs": {
    "MCQQuestion": {
      "description": "Single Multiple Choice Question",
      "properties": {
        "question": {
          "description": "The question text",
          "title": "Question",
          "type": "string"
        },
        "option_a": {
          "description": "Option A",
          "title": "Option A",
          "type": "string"
        },
        "option_b": {
          "description": "Option B",
          "title": "Option B",
          "type": "string"
        },
        "option_c": {
          "description": "Option C",
          "title": "Option C",
          "type": "string"
        },
        "option_d": {
          "description": "Option D",
          "title": "Option D",
          "type": "string"
        },
        "correct_answer": {
          "description": "Correct answer: A, B, C, or D",
          "title": "Correct Answer",
          "type": "string"
        }
      },
      "required": [
        "question",

## 4. Prepare Dataset

In [7]:
from datasets import load_dataset, Dataset, concatenate_datasets
import re
import ast

SYSTEM_PROMPT = """You are an expert MCQ question generator. Given a text passage, generate multiple choice questions.

Output Format:
<questions>
[
  {
    "question": "Question text here?",
    "option_a": "First option",
    "option_b": "Second option",
    "option_c": "Third option",
    "option_d": "Fourth option",
    "correct_answer": "A"
  }
]
</questions>
"""

def prepare_race_dataset(split="test", num_samples=500, config="high"):
    """Convert RACE dataset to MCQ generation format"""
    if config == "both":
        dataset_high = load_dataset("EleutherAI/race", "high", split=split)
        dataset_middle = load_dataset("EleutherAI/race", "middle", split=split)
        dataset = concatenate_datasets([dataset_high, dataset_middle])
    else:
        dataset = load_dataset("EleutherAI/race", config, split=split)
    
    prepared_data = []
    
    for idx, example in enumerate(dataset):
        if idx >= num_samples:
            break
            
        article = example['article']
        problems = ast.literal_eval(example['problems'])
        
        mcq_list = []
        for problem in problems:
            question_text = problem['question']
            options = problem['options']
            answer = problem['answer']
            
            mcq_item = {
                "question": question_text,
                "option_a": options[0] if len(options) > 0 else "",
                "option_b": options[1] if len(options) > 1 else "",
                "option_c": options[2] if len(options) > 2 else "",
                "option_d": options[3] if len(options) > 3 else "",
                "correct_answer": answer
            }
            mcq_list.append(mcq_item)
        
        expected_output = "<questions>\n" + json.dumps(mcq_list, indent=2) + "\n</questions>"
        
        prepared_data.append({
            'prompt': [
                {'role': 'system', 'content': SYSTEM_PROMPT},
                {'role': 'user', 'content': f"Generate MCQ questions from this text:\n\n{article}"}
            ],
            'expected_output': expected_output,
            'num_questions': len(mcq_list)
        })
    
    return Dataset.from_list(prepared_data)

print("Loading and preparing dataset...")
train_dataset = prepare_race_dataset(split="test", num_samples=500, config="high")

print(f"✓ Prepared {len(train_dataset)} training examples")
print(f"\nExample entry:")
print(f"Prompt: {train_dataset[0]['prompt'][1]['content'][:200]}...")
print(f"\nExpected Output: {train_dataset[0]['expected_output'][:300]}...")

Loading and preparing dataset...
✓ Prepared 500 training examples

Example entry:
Prompt: Generate MCQ questions from this text:

The rain had continued for a week and the flood had created a big river which were running by Nancy Brown's farm. As she tried to gather her cows to a higher gr...

Expected Output: <questions>
[
  {
    "question": "What did Nancy try to do before she fell over?",
    "option_a": "Measure the depth of the river",
    "option_b": "Look for a fallen tree trunk",
    "option_c": "Protect her cows from being drowned",
    "option_d": "Run away from the flooded farm",
    "correct_...


## 5. Define Reward Functions

In [8]:
import json
import re

def extract_json_content(text: str) -> str:
    """Extract JSON from <questions> tags"""
    match = re.search(r'<questions>\s*(.+?)\s*</questions>', text, re.DOTALL)
    if match:
        return match.group(1).strip()
    return ""

def format_reward_func(completions, **kwargs) -> list[float]:
    """Reward for proper XML format with <questions> tags"""
    responses = [completion[0]['content'] for completion in completions]
    rewards = []
    for response in responses:
        reward = 0.0
        if '<questions>' in response:
            reward += 0.25
        if '</questions>' in response:
            reward += 0.25
        rewards.append(reward)
    return rewards

def json_validity_reward_func(completions, **kwargs) -> list[float]:
    """Reward for valid JSON structure"""
    responses = [completion[0]['content'] for completion in completions]
    rewards = []
    for response in responses:
        json_content = extract_json_content(response)
        try:
            parsed = json.loads(json_content)
            if isinstance(parsed, list):
                rewards.append(0.5)
            else:
                rewards.append(0.0)
        except:
            rewards.append(0.0)
    return rewards

def mcq_structure_reward_func(completions, **kwargs) -> list[float]:
    """Reward for proper MCQ structure"""
    responses = [completion[0]['content'] for completion in completions]
    rewards = []
    required_fields = ['question', 'option_a', 'option_b', 'option_c', 'option_d', 'correct_answer']
    
    for response in responses:
        json_content = extract_json_content(response)
        try:
            parsed = json.loads(json_content)
            if isinstance(parsed, list) and len(parsed) > 0:
                first_q = parsed[0]
                if all(field in first_q for field in required_fields):
                    rewards.append(1.0)
                else:
                    rewards.append(0.0)
            else:
                rewards.append(0.0)
        except:
            rewards.append(0.0)
    return rewards

def answer_validity_reward_func(completions, **kwargs) -> list[float]:
    """Reward for correct answer format (A, B, C, or D)"""
    responses = [completion[0]['content'] for completion in completions]
    rewards = []
    for response in responses:
        json_content = extract_json_content(response)
        try:
            parsed = json.loads(json_content)
            if isinstance(parsed, list) and len(parsed) > 0:
                valid_answers = sum(1 for q in parsed if q.get('correct_answer', '') in ['A', 'B', 'C', 'D'])
                reward = valid_answers / len(parsed) * 0.5
                rewards.append(reward)
            else:
                rewards.append(0.0)
        except:
            rewards.append(0.0)
    return rewards

def question_count_reward_func(completions, num_questions, **kwargs) -> list[float]:
    """Reward for generating appropriate number of questions"""
    responses = [completion[0]['content'] for completion in completions]
    rewards = []
    for response, expected_count in zip(responses, num_questions):
        json_content = extract_json_content(response)
        try:
            parsed = json.loads(json_content)
            if isinstance(parsed, list):
                actual_count = len(parsed)
                if actual_count == expected_count:
                    rewards.append(0.5)
                elif abs(actual_count - expected_count) <= 1:
                    rewards.append(0.25)
                else:
                    rewards.append(0.0)
            else:
                rewards.append(0.0)
        except:
            rewards.append(0.0)
    return rewards

print("✓ Reward functions defined!")

✓ Reward functions defined!


## 6. Configure GRPO Training

In [9]:
from trl import GRPOConfig, GRPOTrainer

training_args = GRPOConfig(
    use_vllm = True,
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "adamw_8bit",
    logging_steps = 5,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 4,
    num_generations = 4,
    max_prompt_length = 1024,
    max_completion_length = 1024,
    max_steps = 300,
    save_steps = 100,
    max_grad_norm = 0.1,
    report_to = "none",
    output_dir = "outputs_mcq_grpo",
)

print("✓ Training configuration ready!")

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 4
✓ Training configuration ready!


## 7. Train the Model

**Training will take approximately 1-2 hours. Monitor the reward column - it should increase over time.**

In [10]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        format_reward_func,
        json_validity_reward_func,
        mcq_structure_reward_func,
        answer_validity_reward_func,
        question_count_reward_func,
    ],
    args = training_args,
    train_dataset = train_dataset,
)

print("Starting training...")
trainer.train()
print("✓ Training complete!")

[accelerate.utils.other|WARNING]Detected kernel version 4.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Starting training...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 500 | Num Epochs = 3 | Total steps = 300
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 119,734,272 of 3,205,672,960 (3.74% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,kl,rewards / format_reward_func / mean,rewards / format_reward_func / std,rewards / json_validity_reward_func / mean,rewards / json_validity_reward_func / std,rewards / mcq_structure_reward_func / mean,rewards / mcq_structure_reward_func / std,rewards / answer_validity_reward_func / mean,rewards / answer_validity_reward_func / std,rewards / question_count_reward_func / mean,rewards / question_count_reward_func / std
5,0.000000,2.293750,0.435454,420.200000,307.200000,607.200000,0.000000,420.200000,307.200000,607.200000,0.000303,0.500000,0.000000,0.475000,0.065311,0.925000,0.145743,0.212500,0.251576,0.181250,0.166710
10,0.000000,2.475000,0.274843,425.025000,299.000000,626.600000,0.000000,425.025000,299.000000,626.600000,0.004093,0.500000,0.000000,0.493750,0.025000,0.987500,0.050000,0.287500,0.251576,0.206250,0.174649
15,0.000000,2.325000,0.344199,412.950000,268.400000,553.200000,0.012500,411.500006,268.400000,553.200000,0.005258,0.496875,0.012500,0.475000,0.084157,0.950000,0.168313,0.237500,0.247191,0.165625,0.154913
20,0.000000,2.307812,0.488550,418.687500,289.400000,622.600000,0.000000,418.687500,289.400000,622.600000,0.000575,0.500000,0.000000,0.450000,0.130623,0.900000,0.261245,0.254688,0.241201,0.203125,0.158550
25,0.000000,2.336875,0.464764,430.087500,300.800000,701.200000,0.012500,424.431671,300.800000,652.800000,0.005259,0.496875,0.012500,0.468750,0.109157,0.925000,0.236626,0.290000,0.247191,0.156250,0.167253
30,0.000000,2.296875,0.521641,422.975000,288.200000,675.600000,0.000000,422.975000,288.200000,675.600000,0.001246,0.493750,0.025000,0.462500,0.099468,0.912500,0.207756,0.300000,0.249852,0.128125,0.164060
35,0.000000,2.462500,0.440523,430.800000,302.400000,645.200000,0.000000,430.800000,302.400000,645.200000,0.003871,0.500000,0.000000,0.468750,0.109157,0.937500,0.218313,0.387500,0.203494,0.168750,0.192911
40,0.000000,2.475000,0.293438,430.725000,314.800000,576.200000,0.000000,430.725000,314.800000,576.200000,0.004045,0.500000,0.000000,0.475000,0.100000,0.950000,0.200000,0.456250,0.124468,0.093750,0.145084
45,0.000000,2.600000,0.273803,417.350000,261.200000,585.200000,0.012500,414.079169,261.200000,568.000000,0.004160,0.496875,0.012500,0.481250,0.075000,0.962500,0.150000,0.475000,0.084157,0.184375,0.157952
50,0.000000,2.687500,0.213986,384.837500,249.800000,534.000000,0.000000,384.837500,249.800000,534.000000,0.005144,0.500000,0.000000,0.487500,0.050000,0.975000,0.100000,0.487500,0.050000,0.237500,0.174878


Unsloth: Input IDs of shape torch.Size([16, 1595]) with length 1595 > the model's max sequence length of 1536.
We shall truncate it ourselves. It's imperative if you correct this issue first.
Unsloth: Input IDs of shape torch.Size([4, 1595]) with length 1595 > the model's max sequence length of 1536.
We shall truncate it ourselves. It's imperative if you correct this issue first.
Unsloth: Input IDs of shape torch.Size([16, 1537]) with length 1537 > the model's max sequence length of 1536.
We shall truncate it ourselves. It's imperative if you correct this issue first.
Unsloth: Input IDs of shape torch.Size([4, 1537]) with length 1537 > the model's max sequence length of 1536.
We shall truncate it ourselves. It's imperative if you correct this issue first.
Unsloth: Input IDs of shape torch.Size([16, 1546]) with length 1546 > the model's max sequence length of 1536.
We shall truncate it ourselves. It's imperative if you correct this issue first.
Unsloth: Input IDs of shape torch.Size([4,

✓ Training complete!


## 8. Save LoRA Adapters

In [12]:
# Save locally
model.save_lora("mcq_grpo_lora")
print("✓ LoRA adapters saved locally to 'mcq_grpo_lora'")

# Save to Hugging Face Hub
# REPLACE WITH YOUR ACTUAL TOKEN - Delete this cell after successful upload for security
HF_TOKEN = ""  # ← Paste your token here

if HF_TOKEN and HF_TOKEN.startswith('hf_') and len(HF_TOKEN) > 10:
    try:
        print("🔄 Uploading to Hugging Face Hub...")
        model.push_to_hub("mohamedashraff22/qwen2.5-3b-mcq-lora", token=HF_TOKEN)
        tokenizer.push_to_hub("mohamedashraff22/qwen2.5-3b-mcq-lora", token=HF_TOKEN)
        print("✅ LoRA adapters successfully pushed to Hugging Face Hub!")
        print("📦 Model available at: https://huggingface.co/mohamedashraff22/qwen2.5-3b-mcq-lora")
        
        # Clear token from memory for security
        HF_TOKEN = None
        print("🔒 Token cleared from memory")
    except Exception as e:
        print(f"❌ Failed to push to HF: {e}")
        print("💡 Tip: Check if repo exists and token has write permissions")
else:
    print("⚠️ Invalid token format. Skipping HF upload.")
    print("📥 Files saved locally to 'mcq_grpo_lora/' - download manually")


✓ LoRA adapters saved locally to 'mcq_grpo_lora'
🔄 Uploading to Hugging Face Hub...


README.md:   0%|          | 0.00/617 [00:00<?, ?B/s]

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...pyuxfqjgw/adapter_model.safetensors:   0%|          | 30.2kB /  479MB            

Saved model to https://huggingface.co/mohamedashraff22/qwen2.5-3b-mcq-lora


Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  /tmp/tmp9zeavcgk/tokenizer.json       :  97%|#########7| 11.1MB / 11.4MB            

✅ LoRA adapters successfully pushed to Hugging Face Hub!
📦 Model available at: https://huggingface.co/mohamedashraff22/qwen2.5-3b-mcq-lora
🔒 Token cleared from memory


## 10. Test Inference - Fine-tuned Model with LoRA

In [14]:
lora_adapter = model.load_lora("mcq_grpo_lora")

print("=" * 80)
print("FINE-TUNED MODEL OUTPUT (With LoRA):")
print("=" * 80)

output_finetuned = model.fast_generate(
    test_prompt,
    sampling_params = sampling_params,
    lora_request = lora_adapter,
)[0].outputs[0].text

print(output_finetuned)
print("\n" + "=" * 80)

FINE-TUNED MODEL OUTPUT (With LoRA):


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|             | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s…

<questions>
[
  {
    "question": "What happened to Nancy after she slipped and hit her head?",
    "option_a": "She fell unconscious and couldn't wake up.",
    "option_b": "She slipped and hit her head on a fallen tree trunk.",
    "option_c": "She gathered her cows and began walking slowly.",
    "option_d": "She was licked by Lizzie and everything was fine.",
    "correct_answer": "A"
  },
  {
    "question": "Who was licking Nancy's face when she came to?",
    "option_a": "Lizzie was licking her face.",
    "option_b": "The flood was licking her face.",
    "option_c": "Her friends were licking her face.",
    "option_d": "Lizzie was licking her face.",
    "correct_answer": "D"
  },
  {
    "question": "How was Nancy able to gather all her strength?",
    "option_a": "She was already strong before the accident.",
    "option_b": "She was helped by her friends.",
    "option_c": "She was inspired by Lizzie.",
    "option_d": "She was licking her face.",
    "correct_answer": "C"


## 11. Parse and Validate Output

In [15]:
def parse_mcq_output(output_text: str) -> dict:
    """Parse and validate MCQ output"""
    json_content = extract_json_content(output_text)
    
    try:
        parsed = json.loads(json_content)
        mcq_list = MCQList(questions=[MCQQuestion(**q) for q in parsed])
        
        return {
            "success": True,
            "data": mcq_list.model_dump(),
            "num_questions": len(mcq_list.questions)
        }
    except Exception as e:
        return {
            "success": False,
            "error": str(e),
            "raw_output": output_text[:500]
        }

result = parse_mcq_output(output_finetuned)

if result["success"]:
    print("✅ Successfully generated and validated MCQ questions!")
    print(f"\nGenerated {result['num_questions']} questions:")
    print(json.dumps(result["data"], indent=2))
else:
    print("❌ Failed to parse output:")
    print(f"Error: {result['error']}")
    print(f"Raw output: {result['raw_output']}")

✅ Successfully generated and validated MCQ questions!

Generated 3 questions:
{
  "questions": [
    {
      "question": "What happened to Nancy after she slipped and hit her head?",
      "option_a": "She fell unconscious and couldn't wake up.",
      "option_b": "She slipped and hit her head on a fallen tree trunk.",
      "option_c": "She gathered her cows and began walking slowly.",
      "option_d": "She was licked by Lizzie and everything was fine.",
      "correct_answer": "A"
    },
    {
      "question": "Who was licking Nancy's face when she came to?",
      "option_a": "Lizzie was licking her face.",
      "option_b": "The flood was licking her face.",
      "option_c": "Her friends were licking her face.",
      "option_d": "Lizzie was licking her face.",
      "correct_answer": "D"
    },
    {
      "question": "How was Nancy able to gather all her strength?",
      "option_a": "She was already strong before the accident.",
      "option_b": "She was helped by her friend

## 12. Test with Large Custom Text

In [16]:
large_text = """Machine learning is a subset of artificial intelligence that provides systems the ability to automatically learn 
and improve from experience without being explicitly programmed. Machine learning focuses on the development of computer programs 
that can access data and use it to learn for themselves. The process of learning begins with observations or data, such as examples, 
direct experience, or instruction, in order to look for patterns in data and make better decisions in the future based on the examples 
that we provide. The primary aim is to allow the computers to learn automatically without human intervention or assistance and adjust 
actions accordingly.

There are several types of machine learning algorithms. Supervised learning algorithms can apply what has been learned in the past to new data 
using labeled examples to predict future events. Starting from the analysis of a known training dataset, the learning algorithm produces an 
inferred function to make predictions about the output values. The system is able to provide targets for any new input after sufficient training.

Unsupervised learning is used when the information used to train is neither classified nor labeled. Unsupervised learning studies how systems 
can infer a function to describe a hidden structure from unlabeled data. The system doesn't figure out the right output, but it explores the 
data and can draw inferences from datasets to describe hidden structures from unlabeled data."""

large_prompt = tokenizer.apply_chat_template([
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": f"Generate MCQ questions from this text:\n\n{large_text}"}
], tokenize=False, add_generation_prompt=True)

print("=" * 80)
print("GENERATING MCQs FROM LARGE TEXT:")
print("=" * 80)

output_large = model.fast_generate(
    large_prompt,
    sampling_params = SamplingParams(
        temperature = 0.7,
        top_p = 0.9,
        max_tokens = 1536,
    ),
    lora_request = lora_adapter,
)[0].outputs[0].text

print(output_large)
print("\n" + "=" * 80)

result_large = parse_mcq_output(output_large)
if result_large["success"]:
    print(f"\n✅ Successfully generated {result_large['num_questions']} questions from large text!")
    
    with open('generated_mcqs.json', 'w') as f:
        json.dump(result_large["data"], f, indent=2)
    print("📁 Saved to 'generated_mcqs.json'")
else:
    print(f"\n❌ Parsing failed: {result_large['error']}")

GENERATING MCQs FROM LARGE TEXT:


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|             | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s…

<questions>
[
  {
    "question": "What is the primary goal of machine learning?",
    "option_a": "To provide systems the ability to automatically learn and improve from experience without being explicitly programmed",
    "option_b": "To make predictions based on existing data without learning from past examples",
    "option_c": "To allow the computers to learn automatically without human intervention and adjust actions accordingly",
    "option_d": "To teach computers to learn by themselves without human intervention or assistance",
    "correct_answer": "C"
  },
  {
    "question": "What type of machine learning algorithms can apply past learning to new data to predict future events?",
    "option_a": "Unsupervised learning",
    "option_b": "Supervised learning",
    "option_c": "Reinforcement learning",
    "option_d": "Clustering learning",
    "correct_answer": "B"
  },
  {
    "question": "What is unsupervised learning used for?",
    "option_a": "To apply past learning to ne

## 13. Inference Helper Function

In [17]:
def generate_mcqs_from_text(text: str, num_questions: int = None) -> dict:
    """Generate MCQ questions from input text"""
    user_content = f"Generate MCQ questions from this text:\n\n{text}"
    if num_questions:
        user_content += f"\n\nGenerate exactly {num_questions} questions."
    
    prompt = tokenizer.apply_chat_template([
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_content}
    ], tokenize=False, add_generation_prompt=True)
    
    output = model.fast_generate(
        prompt,
        sampling_params = SamplingParams(
            temperature = 0.7,
            top_p = 0.9,
            max_tokens = 1536,
        ),
        lora_request = lora_adapter,
    )[0].outputs[0].text
    
    return parse_mcq_output(output)

test_text = """Python is a high-level, interpreted programming language known for its simplicity and readability. 
It was created by Guido van Rossum and first released in 1991. Python supports multiple programming paradigms, 
including procedural, object-oriented, and functional programming."""

result = generate_mcqs_from_text(test_text, num_questions=3)

if result["success"]:
    print("✅ Generated MCQs:")
    for i, q in enumerate(result["data"]["questions"], 1):
        print(f"\n{i}. {q['question']}")
        print(f"   A) {q['option_a']}")
        print(f"   B) {q['option_b']}")
        print(f"   C) {q['option_c']}")
        print(f"   D) {q['option_d']}")
        print(f"   Correct Answer: {q['correct_answer']}")
else:
    print(f"❌ Error: {result['error']}")

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|             | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s…

✅ Generated MCQs:

1. What does Python support?
   A) Procedural programming only
   B) Object-oriented programming only
   C) Functional programming only
   D) Procedural, object-oriented, and functional programming
   Correct Answer: D

2. Who created Python?
   A) Guido van Rossum
   B) Bjarne Stroustrup
   C) James Gosling
   D) Dennis Ritchie
   Correct Answer: A

3. When was Python first released?
   A) 1990
   B) 1991
   C) 1992
   D) 1993
   Correct Answer: B


## 14. Batch Processing - Generate MCQs from Multiple Texts

In [18]:
def batch_generate_mcqs(texts: List[str], output_file: str = "batch_mcqs.json") -> dict:
    """Generate MCQs from multiple text passages"""
    results = []
    
    for idx, text in enumerate(texts):
        print(f"Processing text {idx+1}/{len(texts)}...")
        result = generate_mcqs_from_text(text)
        
        results.append({
            "text_id": idx,
            "text_preview": text[:100] + "...",
            "success": result["success"],
            "questions": result.get("data", {}).get("questions", []),
            "num_questions": result.get("num_questions", 0)
        })
    
    with open(output_file, 'w') as f:
        json.dump(results, f, indent=2)
    
    print(f"\n✅ Batch processing complete! Saved to '{output_file}'")
    return results

sample_texts = [
    """Artificial Intelligence (AI) refers to the simulation of human intelligence in machines that are programmed 
    to think like humans and mimic their actions. The term may also be applied to any machine that exhibits traits 
    associated with a human mind such as learning and problem-solving.""",
    
    """Deep learning is part of a broader family of machine learning methods based on artificial neural networks with 
    representation learning. Learning can be supervised, semi-supervised or unsupervised. Deep learning architectures 
    such as deep neural networks have been applied to fields including computer vision and natural language processing."""
]

batch_results = batch_generate_mcqs(sample_texts)

print(f"\nBatch Summary:")
for result in batch_results:
    status = "✅" if result["success"] else "❌"
    print(f"{status} Text {result['text_id']}: {result['num_questions']} questions generated")

Processing text 1/2...


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|             | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s…

Processing text 2/2...


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|             | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s…


✅ Batch processing complete! Saved to 'batch_mcqs.json'

Batch Summary:
✅ Text 0: 3 questions generated
✅ Text 1: 3 questions generated


## Summary

This notebook demonstrates:
1. ✅ Fine-tuning Qwen2.5-3B-Instruct with GRPO for MCQ generation
2. ✅ Using Pydantic models for structured output validation
3. ✅ Training on RACE dataset format
4. ✅ Saving LoRA adapters (not merged with base model)
5. ✅ Loading adapters from Hugging Face Hub
6. ✅ Generating MCQs from custom text passages
7. ✅ Batch processing multiple texts
8. ✅ Exporting results to JSON files

**Next Steps:**
- Upload your LoRA adapters to Hugging Face Hub
- Test with your own book chapters
- Adjust reward functions for better quality
- Extend training for improved performance

In [19]:
# Zip the adapter folder
import shutil
import os

# Create zip file
shutil.make_archive('mcq_grpo_lora', 'zip', 'mcq_grpo_lora')

# Get file size
zip_size = os.path.getsize('mcq_grpo_lora.zip') / (1024*1024)  # MB
print(f"✓ Created mcq_grpo_lora.zip ({zip_size:.2f} MB)")
print("📥 Download the file from the file browser in the left sidebar")
print("\nSteps:")
print("1. Click the folder icon (📁) in the left sidebar")
print("2. Find 'mcq_grpo_lora.zip' in the file list")
print("3. Right-click → Download")


✓ Created mcq_grpo_lora.zip (179.51 MB)
📥 Download the file from the file browser in the left sidebar

Steps:
1. Click the folder icon (📁) in the left sidebar
2. Find 'mcq_grpo_lora.zip' in the file list
3. Right-click → Download
